In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')
from visualize import visualize_svm as Visualize

#
sys.path.append('../..')
from svm import svm as SVM


Autosaving every 180 seconds


<ipython-input-1-a90fe7f1d8b3>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#################################################
############# HELPER FUNCTIONS ##################
#################################################
# 
import scipy
from statsmodels.stats.multitest import multipletests

#
def load_supersessions(animal_id, lockout):
    fname = os.path.join('/media/cat/4TBSSD/yuki/',
                         animal_id,
                         'super_sessions',
                         'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    trials = np.vstack(data['trials'])
    random = np.vstack(data['random'])
    
    return trials, random

# 
def filter_trace(trace, smooth_window=30):

    box = np.ones(smooth_window)/smooth_window
    trace_smooth = np.convolve(trace, box, mode='valid')

    return trace_smooth


def compute_significance(data):

    #
    significance = 0.05

    #
    sig = []
    for k in range(data.shape[0]):

        #
        res = scipy.stats.ttest_1samp(data[k], 0.5)

        sig.append(res[1])


    sig_save = np.array(sig).copy()

    # multiple hypothesis test Benjamini-Hockberg
    temp = np.array(sig)
    temp2 = multipletests(temp, 
                          alpha=significance, 
                          method='fdr_bh')
    sig = temp2[1]

    # find earliest significant;
    edt = 0
    for k in range(sig.shape[0]-1,0,-1):
        if sig[k]<=significance:
            edt = k+1
        else:
            break

    # 
    edtsec = -(sig.shape[0]-edt)/30.

    # 
    return edtsec

In [6]:
################################################################
#### SUBSAMPLE 3SEC LOCKOUT TRIALS TO MATCH 30SEC LOCKOUTS #####
################################################################
# 
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
lockout1 = 9

#    
for animal_id in animal_ids:
    # animal_id = 'AQ2'

    print ("animal id: ", animal_id)
    # load 30sec lockout 
    trials30, random30 = load_supersessions(animal_id, lockout1)
    print ("trials30: ", trials30.shape)

    # load 3sec lockouts
    lockout2 = 0  # the 3sec lockout data is saved as 0seclockout
    trials3, random3 = load_supersessions(animal_id, lockout2)
    print ("trials3: ", trials3.shape)

    # subsample 3sec lockout data
    idx = np.int32(np.linspace(0,trials3.shape[0]-1,trials30.shape[0]))
    trials3_sub = trials3[idx].transpose(0,2,1)
    random3_sub = random3[idx].transpose(0,2,1)
    print ("trials3_sub: ", trials3_sub.shape)

    # 
    svm = SVM.PredictSVMChoiceSuperSession()
    svm.xvalidation = 10
    svm.parallel = True

    # window parameters
    svm.window = 30              # seconds of prediction window, backwards in time in seconds; e.g. -30sec ... +30sec
    svm.lockout_window = 10      # time for locking out other pulls 
    svm.sliding_window = 30      # number of frames in sliding window

    # pca params
    svm.pca_flag = True
    svm.pca_var = 0.95        # this is default for now for nComp = 20

    # svm parameters
    svm.xvalidation = 10      # KFold xvalidation step
    svm.data_split = 0.8      # split 80/20 data for training
    svm.method = 'sigmoid'    # method used for computing SVM

    # run-time parameters
    svm.parallel = True
    svm.n_cores = svm.xvalidation
    svm.min_trials = 30
    svm.overwrite = True
    svm.gpu_flag = False

    #
    fname_out = os.path.join('/media/cat/4TBSSD/yuki/',
                             animal_id,
                             'super_sessions',
                             'alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats_super_res_subsample_'+
                             str(lockout1)+'matched_lockout.npz')

    #
    svm.predict3(trials3_sub, random3_sub, fname_out)



animal id:  IA1
trials30:  (741, 1800, 16)
trials3:  (1351, 1800, 16)
trials3_sub:  (741, 16, 1800)
   start:  0  end:  200  /  741
   start:  50  end:  250  /  741
   start:  100  end:  300  /  741
   start:  150  end:  350  /  741
   start:  200  end:  400  /  741
   start:  250  end:  450  /  741
   start:  300  end:  500  /  741
   start:  350  end:  550  /  741
   start:  400  end:  600  /  741
   start:  450  end:  650  /  741
   start:  500  end:  700  /  741
   start:  550  end:  750  /  741
   start:  600  end:  800  /  741
   start:  650  end:  850  /  741
   start:  700  end:  900  /  741
animal id:  IA2
trials30:  (1027, 1800, 16)
trials3:  (1704, 1800, 16)
trials3_sub:  (1027, 16, 1800)
   start:  0  end:  200  /  1027
   start:  50  end:  250  /  1027
   start:  100  end:  300  /  1027
   start:  150  end:  350  /  1027
   start:  200  end:  400  /  1027
   start:  250  end:  450  /  1027
   start:  300  end:  500  /  1027
   start:  350  end:  550  /  1027
   start:  400

In [5]:
############################################
###### COMPUTE FIRST DECODING TIMES ########
############################################
root_dir = '/media/cat/4TBSSD/yuki'

#
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA1']
lockout1 = 15
window = 50
for animal_id in animal_ids:
    edts = []

    # 
    for n in range(0, 10000, window):

        fname = os.path.join(root_dir,
                             animal_id,
                             'super_sessions',
                             'alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats_super_res_subsample_'
                             +str(lockout1)+'matched_lockout_'+
                             str(n)+'_super_res.npz')

        try:
            data = np.load(fname,
                      allow_pickle=True)
        except:
            print ("Done ...  ",animal_id)
            break

        acc = data['accuracy']

        # 
        acc2 = []
        for k in range(acc.shape[1]):
            acc2.append(filter_trace(acc[:,k]))

        acc2 = np.vstack(acc2).T
        #plt.plot(acc2.mean(1))
        edt = compute_significance(acc2[:900])
        
        # must add 1 sec because of SVM works in 1 sec chunks
        edt = edt+1
        if edt>0:
            edt=0
            
            
        edts.append(edt)

    #
    np.savez('/home/cat/'+animal_id+"_edts_locked_out_subsampled_"+str(lockout1)+".npz", 
             all_res_continuous = edts)

print ("... DONE ALL...")

Done ...   IA1
Done ...   IA2
Done ...   IA3
Done ...   IJ1
Done ...   IJ2
Done ...   AQ2


In [5]:
###################################################################
####### PLOT BOX PLOTS: ALL vs. ALL-subsampled vs LOCKED OUT ######
###################################################################
import pandas as pd
import matplotlib.patches as mpatches
import scipy.stats as stats

# 
def plot_edt_distributions_box_plots(animal_id,ctr,lockout):

    #
    codes = ['all', 'all-subsampled','lockedout']
    clrs_local = ['red','green','yellow']

    #
    vis = Visualize.Visualize()
    
    # fnames = [
    #     '/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time.npz',
    #     '/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_0.3hzFilter.npz',
    # ]
    
    legend_size = 22
        
    # 
    shift= 15
    linestyles = ['-','-','-','-','-','-','-','-']

    #titles = ['all','lockout','concatenated']
    pvals = [0.05,0.01,0.001,0.0001,0.00001]
    edts = []
    yvals = []

    # 
    fname1 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_concatenated.npz')
    fname2 = os.path.join('/home/cat/'+animal_id+'_edts_locked_out_subsampled_'+str(lockout)+'.npz')
    fname3 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_lockedout.npy')
    
    #
    res1 = np.load(fname1, allow_pickle=True)['all_res_continuous']
    edts.append(res1)
    yvals.append(res1[1])

    #
    res2 = np.load(fname2, allow_pickle=True)['all_res_continuous']
    edts.append(res2)
    yvals.append(res2[1])

    # print differences between means
    print (animal_id, "  diff in means: ", np.mean(res1) - np.mean(res2))
    
    
    #
    res3 = np.load(fname3, allow_pickle=True)
    for k in range(len(res3)):
        if res3[k][0]==lockout:
            res3 = np.hstack(res3[k][1])
            break
    
    edts.append(res3)
    yvals.append(np.arange(res3.shape[0]))
    
    # 
    my_dict = dict(No_lockout = edts[0], 
                   No_lockout_sub = edts[1],
                   Lockout = edts[2],
                 )
    
    # 
    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    ######################################################### 
    ################## SCATTER PLOTS ########################
    ######################################################### 
    for i,d in enumerate(data):
        y = data[d]
        x = np.random.normal(i+1, 0.04, len(y))
        plt.plot(x, y, 
                 mfc =clrs_local[i], 
                 mec='k', 
                 ms=7, 
                 marker="o", 
                 linestyle="None",
                 alpha=.8

                )


    patches = []
    for c,code in enumerate(codes):
        patches.append(mpatches.Patch(color=clrs_local[c], label=code))

    ######################################################### 
    ###################### BOX PLOTS ########################
    ######################################################### 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                flierprops=flierprops)

    temp = np.hstack(yvals)
    ymax = np.max(temp)

    #
    plt.xlim(0.5, 4)

    #
    plt.plot([0,6.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-5,-5],'--',linewidth=3,c='grey',alpha=.5)

    # ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
    ###########################################
    patches = []
    for p in [1,2]:
        res = stats.ks_2samp(edts[0], edts[p])
        print ("res: 0 vs ",p , res)
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color=clrs_local[p-1],label=label_))

    ###################################
    res = stats.ks_2samp(edts[1], edts[2])
    print ("res: 1 vs 2 ", res)
    label_ = ''
    for k in range(len(pvals)):
        if res[1]<pvals[k]:
            label_ = label_ + "*"
        else:
            break

    patches.append(mpatches.Patch(color=clrs_local[2], label=label_))
    
    plt.xticks(rotation=45)
    #
    plt.legend(handles=patches,
               fontsize=15)
    plt.ylim(-15,0)


#############################################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA1']
lockout=15

#
ctr=0
fig=plt.figure(figsize=(20,5))
diffs = []
for animal_id in animal_ids:
    ax=plt.subplot(1,6,ctr+1)
    
    diff = plot_edt_distributions_box_plots(animal_id,
                                     ctr,
                                     lockout)
    ctr+=1
    
plt.suptitle("Lockout "+str(lockout))
#######################################
if False:
    plt.savefig('/home/cat/filtered_EDTs.svg')
    plt.close()
else:
    plt.show()

IA1   diff in means:  -1.9208805031446539
res: 0 vs  1 KstestResult(statistic=0.6113207547169811, pvalue=0.001712096640566485)
res: 0 vs  2 KstestResult(statistic=0.8056603773584906, pvalue=4.238345720830594e-06)
res: 1 vs 2  KstestResult(statistic=0.2, pvalue=0.9944575548290717)
IA2   diff in means:  -0.932323232323232
res: 0 vs  1 KstestResult(statistic=0.2727272727272727, pvalue=0.5374816655062998)
res: 0 vs  2 KstestResult(statistic=0.6666666666666666, pvalue=0.0006469882104048752)
res: 1 vs 2  KstestResult(statistic=0.45454545454545453, pvalue=0.21147608841107293)
IA3   diff in means:  -3.840447154471545
res: 0 vs  1 KstestResult(statistic=1.0, pvalue=1.3423269237144986e-05)
res: 0 vs  2 KstestResult(statistic=1.0, pvalue=1.3423269237144986e-05)
res: 1 vs 2  KstestResult(statistic=1.0, pvalue=0.028571428571428577)
IJ1   diff in means:  -3.3049999999999997
res: 0 vs  1 KstestResult(statistic=0.975, pvalue=9.821904319973207e-06)
res: 0 vs  2 KstestResult(statistic=0.975, pvalue=9.82

In [155]:
############################################
############################################
############################################
fname = '/media/cat/4TBSSD/yuki/IA1/first_decoding_time_0.3hzFilter.npz'
data = np.load(fname)
names = data['all_session_names']
print (names[idx_best])


['IA1pm_Feb23_30Hz' 'IA1pm_Feb19_30Hz' 'IA1pm_Feb16_30Hz'
 'IA1pm_Feb17_30Hz' 'IA1pm_Apr25_Week3_30Hz']


In [156]:
#############################################
#############################################
#############################################
session_name = 'IA1pm_Feb23_30Hz'

trials = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_trial_ROItimeCourses_30sec_pca_0.95_.3hzfiltered.npy')
trials2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name+'_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
random = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_random_ROItimeCourses_30sec_pca_0.95_.3hzfiltered.npy')
random2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_random_ROItimeCourses_30sec_pca_0.95.npy')

print (trials.shape)
print (random.shape)

#
t = np.arange(trials.shape[2])/30.-30

#
plt.figure(figsize=(20,10))
ax1=plt.subplot(2,1,1)
plt.plot(t,trials2[0,0].T,c='blue',alpha=1,label='unfiltered')
plt.plot(t,trials[0,0].T,c='green',alpha=1,label='filtered')
plt.legend(fontsize=20)
plt.plot([-30,30], [0,0],'--',linewidth=2,c='grey',alpha=.5)
plt.plot([0,0],[-10,10],'--',linewidth=2,c='grey',alpha=.5)
plt.xlabel("Time (sec)")
plt.ylabel("Trial")
plt.xlim(-30,30)
plt.ylim(-10,10)    
#
ax1=plt.subplot(2,1,2)
plt.plot(t,random2[0,0].T,c='red',alpha=1,label='unfiltered')
plt.plot(t,random[0,0].T,c='green',alpha=1,label='filtered')
plt.legend(fontsize=20)
plt.plot([-30,30], [0,0],'--',linewidth=2,c='grey',alpha=.5)
plt.plot([0,0],[-10,10],'--',linewidth=2,c='grey',alpha=.5)
plt.xlabel("Time (sec)")
plt.ylabel("Random")

if False:
    plt.savefig('/home/cat/trials.svg')
    plt.close()
plt.show()


(81, 8, 1801)
(76, 8, 1801)


In [157]:
############################################
########## QUICK AND DIRTY CHECKS ##########
############################################
#session_name = 'IA1pm_Feb23_30Hz'
fnames = [
    '/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_'+session_name+'code_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz',
    '/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_'+session_name+'code_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30_.3HzFilter.npz'
]

# 
fig=plt.figure()
ax=plt.subplot(111)
for ctr,fname in enumerate(fnames):
    data = np.load(fname)
    predictions = data['predictions']
    print ("n_trials: ", predictions.shape[1])
    acc = data['accuracy']
    length = 1801-acc.shape[0]
    acc = np.vstack(
                     (np.zeros((length,10), dtype=np.float32), 
                      acc))
    print (acc.shape)
    t = np.arange(acc.shape[0])/30-30
    mean = acc.mean(1)
    std = np.std(acc,1)
    plt.plot(t,mean)
    ax.fill_between(t, mean+std,
                    mean-std,
                    alpha=.2)

plt.plot([-30,30],[0.5,0.5],'--',c='grey')
plt.ylim(0,1)
plt.legend()
plt.show()


####
# plt.figure()
# d1 = np.load(fname)
# acc1 = d1['accuracy']
# print (acc1.shape)

# plt.plot(acc1.mean(1), label='all freq')

# d2 = np.load(fname_filtered)
# acc2 = d2['accuracy']
# print (acc2.shape)

# plt.plot(acc2.mean(1), label='<0.3hz')
# plt.legend()
# plt.show()

No handles with labels found to put in legend.


n_trials:  157
(1801, 10)
n_trials:  157
(1801, 10)


In [42]:
fname = '/media/cat/4TBSSD/yuki/IA1/first_decoding_time_0.3hzFilter.npz'
data=np.load(fname)

sessions = data['all_session_names']
print (sessions.shape)

(53,)
